In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE243348"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE243348"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE243348.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE243348.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE243348.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Longitudinal gene expression profiling of self-collected blood samples in COVID-19+ and healthy participants"
!Series_summary	"Longitudinal cohort: 773 host response genes were profiled in previously vaccinated (n=16) and unvaccinated (n=14) COVID-19+ participants along with 5 healthy uninfected controls across a 2-week observational window"
!Series_summary	"Single timepoint cohort: 773 host response genes were profiled in 6 healthy uninfected participants"
!Series_overall_design	"Longitudinal cohort: 30 COVID-19+ and 5 uninfected participants were asked perform self-collection and stabilization of capillary blood using a novel technology (homeRNA) every other day for two weeks (7 longtiudinal timepoints per participant). Temporal kinetics of 773 immune genes were profiled using the nCounter direct digital counting of native mRNA."
!Series_overall_design	"Single timepoint cohort: 6 healthy uninfected participants were asked perform self-collection

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# "773 host response genes were profiled using the nCounter direct digital counting of native mRNA"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (COVID-19 status) is in row 0
trait_row = 0

# Age is in row 3
age_row = 3

# Gender is in row 2
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value: str) -> int:
    """
    Convert COVID-19 status to binary (0 for healthy, 1 for COVID-19+)
    """
    if not isinstance(value, str):
        return None
    
    value_lower = value.lower()
    if 'covid-19+' in value_lower:
        return 1
    elif 'healthy' in value_lower:
        return 0
    return None

def convert_age(value: str) -> float:
    """
    Convert age values to continuous numeric values
    """
    if not isinstance(value, str):
        return None
    
    try:
        # Extract the number after the colon
        parts = value.split(': ')
        if len(parts) > 1:
            return float(parts[1])
        return None
    except:
        return None

def convert_gender(value: str) -> int:
    """
    Convert gender to binary (0 for female, 1 for male)
    """
    if not isinstance(value, str):
        return None
    
    value_lower = value.lower()
    if 'female' in value_lower:
        return 0
    elif 'male' in value_lower:
        return 1
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                             is_gene_available=is_gene_available, is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Sample characteristics were provided in the previous step
    # Create a properly structured DataFrame where each row is a characteristic
    # and columns represent different samples
    
    # First, create an empty DataFrame with the sample characteristics as rows
    clinical_data = pd.DataFrame(index=range(8))  # 8 rows for the characteristics
    
    # Add sample characteristics as rows
    sample_chars = {
        0: ['disease status: COVID-19+', 'disease status: Healthy uninfected'],
        1: ['participant id: CB0101', 'participant id: CB0102', 'participant id: CB0104', 'participant id: CB0106', 'participant id: CB0107', 'participant id: CB0111', 'participant id: CB0112', 'participant id: CB0113', 'participant id: CB0115', 'participant id: CB0116', 'participant id: CB0117', 'participant id: CB0118', 'participant id: CB0119', 'participant id: CB0120', 'participant id: CB0121', 'participant id: CB0122', 'participant id: CB0123', 'participant id: CB0124', 'participant id: CB0125', 'participant id: CB0128', 'participant id: CB0129', 'participant id: CB0130', 'participant id: CB0131', 'participant id: CB0132', 'participant id: CB0133', 'participant id: CB0134', 'participant id: CB0135', 'participant id: CB0136', 'participant id: CB0138', 'participant id: CB0139'],
        2: ['Sex: female', 'Sex: male'],
        3: ['age: 44', 'age: 29', 'age: 51', 'age: 32', 'age: 27', 'age: 30', 'age: 41', 'age: 43', 'age: 34', 'age: 60', 'age: 24', 'age: 36', 'age: 33', 'age: 53', 'age: 31', 'age: 59', 'age: 40', 'age: 65', 'age: 37', 'age: 39', 'age: 58', 'age: 42', 'age: 28', 'age: 38'],
        4: ['covid-19 vaccination history: unvaccinated', 'covid-19 vaccination history: vaccinated', 'covid-19 vaccination history: partial'],
        5: ['day post symptom onset: 10', 'day post symptom onset: 13', 'day post symptom onset: 15', 'day post symptom onset: 17', 'day post symptom onset: 19', 'day post symptom onset: 21', 'day post symptom onset: 23', 'day post symptom onset: 9', 'day post symptom onset: 11', 'day post symptom onset: 8', 'day post symptom onset: 12', 'day post symptom onset: 14', 'day post symptom onset: 16', 'day post symptom onset: 18', 'day post symptom onset: 20', 'day post symptom onset: 27', 'day post symptom onset: 25', 'day post symptom onset: 5', 'day post symptom onset: 7', 'day post symptom onset: 6', 'day post symptom onset: 22', 'day post symptom onset: 24', 'day post symptom onset: 26', 'day post symptom onset: 28', 'study day: 1', 'study day: 3', 'study day: 5', 'study day: 7', 'study day: 9', 'study day: 11'],
        6: ['study day: 1', 'study day: 4', 'study day: 6', 'study day: 8', 'study day: 10', 'study day: 12', 'study day: 14', 'study day: 3', 'study day: 5', 'study day: 9', 'study day: 11', 'study day: 13', 'study day: 7', 'study day: 15', 'ncounter host response codeset: V1.0', 'ncounter host response codeset: V1.1'],
        7: ['ncounter host response codeset: V1.0', 'ncounter host response codeset: V1.1', None]
    }
    
    # Populate the DataFrame with the sample characteristics
    for idx, values in sample_chars.items():
        for val in values:
            # Create a new column for each unique value
            col_name = f"sample_{len(clinical_data.columns)}"
            clinical_data[col_name] = None
            clinical_data.at[idx, col_name] = val
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview and save the data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'sample_0': [1.0, nan, nan], 'sample_1': [0.0, nan, nan], 'sample_2': [nan, nan, nan], 'sample_3': [nan, nan, nan], 'sample_4': [nan, nan, nan], 'sample_5': [nan, nan, nan], 'sample_6': [nan, nan, nan], 'sample_7': [nan, nan, nan], 'sample_8': [nan, nan, nan], 'sample_9': [nan, nan, nan], 'sample_10': [nan, nan, nan], 'sample_11': [nan, nan, nan], 'sample_12': [nan, nan, nan], 'sample_13': [nan, nan, nan], 'sample_14': [nan, nan, nan], 'sample_15': [nan, nan, nan], 'sample_16': [nan, nan, nan], 'sample_17': [nan, nan, nan], 'sample_18': [nan, nan, nan], 'sample_19': [nan, nan, nan], 'sample_20': [nan, nan, nan], 'sample_21': [nan, nan, nan], 'sample_22': [nan, nan, nan], 'sample_23': [nan, nan, nan], 'sample_24': [nan, nan, nan], 'sample_25': [nan, nan, nan], 'sample_26': [nan, nan, nan], 'sample_27': [nan, nan, nan], 'sample_28': [nan, nan, nan], 'sample_29': [nan, nan, nan], 'sample_30': [nan, nan, nan], 'sample_31': [nan, nan, nan], 'sample_32': [nan, nan, 0.

/tmp/ipykernel_75088/1254858355.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  clinical_data[col_name] = None
/tmp/ipykernel_75088/1254858355.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  clinical_data[col_name] = None
/tmp/ipykernel_75088/1254858355.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = 

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE243348/GSE243348_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE243348/GSE243348_series_matrix.txt.gz
Found the matrix table marker at line 69
Gene data shape: (773, 237)
First 20 gene/probe identifiers:
['ACE', 'ACKR2', 'ACKR3', 'ACKR4', 'ACOX1', 'ACSL1', 'ACSL3', 'ACSL4', 'ACVR1', 'ADAR', 'ADGRE5', 'ADGRG3', 'ADORA2A', 'AGT', 'AHR', 'AIF1', 'AIM2', 'AKT1', 'AKT2', 'AKT3']


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers
# These identifiers appear to be standard human gene symbols (official gene symbols)
# Examples like ACE, ACKR2, AKT1, etc. are recognized human gene symbols
# No mapping is required as they are already in the correct format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in gene expression data
try:
    # Normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # 2. Attempt to load clinical data and link with genetic data
    try:
        # Load clinical data file saved in Step 2
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        
        # Inspect the clinical data structure
        print("Clinical data columns:", clinical_df.columns[:5], "..." if len(clinical_df.columns) > 5 else "")
        
        # Check for sparsity in clinical data
        sparsity = clinical_df.isna().sum().sum() / (clinical_df.shape[0] * clinical_df.shape[1])
        print(f"Clinical data sparsity: {sparsity:.2%} missing values")
        
        # Count non-NA values in each row of clinical data
        non_na_counts = clinical_df.notna().sum(axis=1)
        print(f"Non-NA values per clinical feature: {non_na_counts.tolist()}")
        
        # Since the clinical data has too many NaN values (as observed in Step 2),
        # and does not contain proper trait information, we cannot link it effectively
        is_trait_available = False
        print("Cannot proceed with linking due to insufficient clinical data (mostly NaN values).")
        
        # 5. Validate and save cohort info - mark as not usable due to lack of trait data
        is_biased = True  # Since we can't even analyze trait distribution
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=is_trait_available,
            is_biased=is_biased,
            df=pd.DataFrame(),  # Empty dataframe since we don't have linked data
            note="Gene expression data available but clinical data contains too many missing values for effective linking."
        )
        
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        is_trait_available = False
        
        # Validate with proper values for unavailable trait data
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=is_trait_available,
            is_biased=True,
            df=pd.DataFrame(),
            note=f"Error processing clinical data: {str(e)}"
        )
    
except Exception as e:
    print(f"Error in gene data processing: {e}")
    
    # Log the error and mark the dataset as unusable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note=f"Error during gene data normalization: {str(e)}"
    )

Gene data shape after normalization: (758, 237)


Normalized gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE243348.csv
Loaded clinical data with shape: (3, 110)
Clinical data columns: Index(['sample_0', 'sample_1', 'sample_2', 'sample_3', 'sample_4'], dtype='object') ...
Clinical data sparsity: 91.52% missing values
Non-NA values per clinical feature: [2, 24, 2]
Cannot proceed with linking due to insufficient clinical data (mostly NaN values).
Abnormality detected in the cohort: GSE243348. Preprocessing failed.
